<a href="https://colab.research.google.com/github/danielscabar/arxiv_text_classification/blob/main/Arxiv_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://medium.com/@JyotsnaPyarasani/building-a-text-classification-system-for-news-articles-a-comprehensive-guide-10a99e8e862d

https://info.arxiv.org/help/api/user-manual.html

https://lukasschwab.me/arxiv.py/arxiv.html#arxivpy

# Data Loading

In [ ]:
!pip install arxiv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.5 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=9a29c6d262addf01444f097a39a151120fa02189e4d6e51050f65078b5af461d
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [ ]:
import arxiv

# Construct the default API client.
client = arxiv.Client()

# Search for the 10 most recent articles matching the keyword "quantum."
search = arxiv.Search(
  query = "quantum",
  max_results = 10,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

results = client.results(search)

In [ ]:
results

In [ ]:
classes_names = ['cs', 'eco', 'eese', 'math', 'phy','bio', 'fin','stats']

# Data Cleaning and Preprocessing

# Feature Engineering

# Model Training

# Model Evaluation

#Building an Ensemble Model

# Model Deployment

# Creating a Streamlit App